# SDE Work-Precision Diagrams

In this notebook we will run some simple work-precision diagrams for the SDE integrators. These problems are additive and diagonal noise SDEs which can utilize the specialized Rossler methods. These problems are very well-behaved, meaning that adaptive timestepping should not be a significant advantage (unlike more difficult and realistic problems). Thus these tests will measure both the efficiency gains of the Rossler methods along with the overhead of adaptivity.

In [1]:
using StochasticDiffEq, Plots, DiffEqDevTools, DiffEqProblemLibrary
gr()

Plots.GRBackend()

In this notebook, the error that will be measured is the strong error. The strong error is defined as

$$ E = \mathbb{E}[Y_\delta(t) - Y(t)] $$

where $Y_\delta$ is the numerical approximation to $Y$. This is the same as saying, for a given Wiener trajectory $W(t)$, how well does the numerical trajectory match the real trajectory? Note that this is not how well the mean or other moments match the true mean/variance/etc. (that's the weak error), this is how close the trajectory is to the true trajectory which is a stronger notion. In a sense, this is measuring convergence, rather than just convergence in distribution.

### Additive Noise Problem

\begin{equation}
dX_{t}=\left(\frac{\beta}{\sqrt{1+t}}-\frac{1}{2\left(1+t\right)}X_{t}\right)dt+\frac{\alpha\beta}{\sqrt{1+t}}dW_{t},\thinspace\thinspace\thinspace X_{0}=\frac{1}{2}
\end{equation}
where $\alpha=\frac{1}{10}$ and $\beta=\frac{1}{20}$. Actual Solution:
\begin{equation}
X_{t}=\frac{1}{\sqrt{1+t}}X_{0}+\frac{\beta}{\sqrt{1+t}}\left(t+\alpha W_{t}\right).
\end{equation}


First let's solve this using a system of SDEs, repeating this same problem 4 times.

In [2]:
prob = prob_sde_additivesystem
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]
setups = [Dict(:alg=>SRIW1())
          Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>SRIW1(),:dts=>1.0./5.0.^((1:length(reltols)) + 1),:adaptive=>false)
          Dict(:alg=>SRA1(),:dts=>1.0./5.0.^((1:length(reltols)) + 1),:adaptive=>false)
          Dict(:alg=>SRA1())
          ]
names = ["SRIW1","EM","RKMil","SRIW1 Fixed","SRA1 Fixed","SRA1"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,names=names,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 SRIW1 
 
 
 
 EM 
 
 
 
 RKMil 
 
 
 
 SRIW1 Fixed 
 
 
 
 SRA1 Fixed 
 
 
 
 SRA1

In [3]:
prob = prob_sde_additivesystem
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]
setups = [
          Dict(:alg=>SRA1())
          Dict(:alg=>SRA2())
          Dict(:alg=>SRA3())
          Dict(:alg=>SOSRA())
          Dict(:alg=>SOSRA2())
          ]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 9 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 3.3 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.7 
 
 
 10 
 
 
 - 
 
 
 2.4 
 
 
 10 
 
 
 - 
 
 
 2.1 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 StochasticDiffEq.SRA1 
 
 
 
 StochasticDiffEq.SRA2 
 
 
 
 StochasticDiffEq.SRA3 
 
 
 
 StochasticDiffEq.SOSRA 
 
 
 
 StochasticDiffEq.SOSRA2

Now as a scalar SDE.

In [4]:
prob = prob_sde_additive
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]



setups = [Dict(:alg=>SRIW1())
          Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>SRIW1(),:dts=>1.0./5.0.^((1:length(reltols)) + 1),:adaptive=>false)
          Dict(:alg=>SRA1(),:dts=>1.0./5.0.^((1:length(reltols)) + 1),:adaptive=>false)
          Dict(:alg=>SRA1())
          ]
names = ["SRIW1","EM","RKMil","SRIW1 Fixed","SRA1 Fixed","SRA1"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,names=names,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3.50 
 
 
 10 
 
 
 - 
 
 
 3.25 
 
 
 10 
 
 
 - 
 
 
 3.00 
 
 
 10 
 
 
 - 
 
 
 2.75 
 
 
 10 
 
 
 - 
 
 
 2.50 
 
 
 10 
 
 
 - 
 
 
 2.25 
 
 
 10 
 
 
 - 
 
 
 2.00 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 SRIW1 
 
 
 
 EM 
 
 
 
 RKMil 
 
 
 
 SRIW1 Fixed 
 
 
 
 SRA1 Fixed 
 
 
 
 SRA1

In [5]:
prob = prob_sde_additive
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]
setups = [
          Dict(:alg=>SRA1())
          Dict(:alg=>SRA2())
          Dict(:alg=>SRA3())
          Dict(:alg=>SOSRA())
          Dict(:alg=>SOSRA2())
          ]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 11 
 
 
 10 
 
 
 - 
 
 
 10 
 
 
 10 
 
 
 - 
 
 
 9 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 StochasticDiffEq.SRA1 
 
 
 
 StochasticDiffEq.SRA2 
 
 
 
 StochasticDiffEq.SRA3 
 
 
 
 StochasticDiffEq.SOSRA 
 
 
 
 StochasticDiffEq.SOSRA2

### Diagonal Noise

We will use a 4x2 matrix of indepdendent linear SDEs (also known as the Black-Scholes equation)

\begin{equation}
dX_{t}=\alpha X_{t}dt+\beta X_{t}dW_{t},\thinspace\thinspace\thinspace X_{0}=\frac{1}{2}
\end{equation}
where $\alpha=\frac{1}{10}$ and $\beta=\frac{1}{20}$. Actual Solution:
\begin{equation}
X_{t}=X_{0}e^{\left(\beta-\frac{\alpha^{2}}{2}\right)t+\alpha W_{t}}.
\end{equation}

In [6]:
prob = prob_sde_2Dlinear
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]

setups = [Dict(:alg=>SRIW1())
          Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>SRIW1(),:dts=>1.0./5.0.^((1:length(reltols)) + 1),:adaptive=>false)
          ]
names = ["SRIW1","EM","RKMil","SRIW1 Fixed"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,names=names,error_estimate=:l2)
plot(wp)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 SRIW1 
 
 
 
 EM 
 
 
 
 RKMil 
 
 
 
 SRIW1 Fixed

In [7]:
prob = prob_sde_2Dlinear
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]

setups = [Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 2))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 2))
          Dict(:alg=>SRI())
          Dict(:alg=>SRIW1())
          Dict(:alg=>SRIW2())
          Dict(:alg=>SOSRI())
          Dict(:alg=>SOSRI2())
          ]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 10 
 
 
 - 
 
 
 1.0 
 
 
 10 
 
 
 - 
 
 
 0.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 StochasticDiffEq.EM 
 
 
 
 StochasticDiffEq.RKMil 
 
 
 
 StochasticDiffEq.SRI 
 
 
 
 StochasticDiffEq.SRIW1 
 
 
 
 StochasticDiffEq.SRIW2 
 
 
 
 StochasticDiffEq.SOSRI 
 
 
 
 StochasticDiffEq.SOSRI2

Now just the scalar Black-Scholes

In [8]:
prob = prob_sde_linear
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]

setups = [Dict(:alg=>SRIW1())
          Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>SRIW1(),:dts=>1.0./5.0.^((1:length(reltols)) + 1),:adaptive=>false)
          ]
names = ["SRIW1","EM","RKMil","SRIW1 Fixed"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,names=names,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 SRIW1 
 
 
 
 EM 
 
 
 
 RKMil 
 
 
 
 SRIW1 Fixed

In [9]:
setups = [Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 2))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 2))
          Dict(:alg=>SRI())
          Dict(:alg=>SRIW1())
          Dict(:alg=>SRIW2())
          Dict(:alg=>SOSRI())
          Dict(:alg=>SOSRI2())
          ]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 StochasticDiffEq.EM 
 
 
 
 StochasticDiffEq.RKMil 
 
 
 
 StochasticDiffEq.SRI 
 
 
 
 StochasticDiffEq.SRIW1 
 
 
 
 StochasticDiffEq.SRIW2 
 
 
 
 StochasticDiffEq.SOSRI 
 
 
 
 StochasticDiffEq.SOSRI2

Now a scalar wave SDE:

\begin{equation}
dX_{t}=-\left(\frac{1}{10}\right)^{2}\sin\left(X_{t}\right)\cos^{3}\left(X_{t}\right)dt+\frac{1}{10}\cos^{2}\left(X_{t}\right)dW_{t},\thinspace\thinspace\thinspace X_{0}=\frac{1}{2}
\end{equation}
Actual Solution:
\begin{equation}
X_{t}=\arctan\left(\frac{1}{10}W_{t}+\tan\left(X_{0}\right)\right).
\end{equation}

In [10]:
prob = prob_sde_wave
prob = SDEProblem(prob.f,prob.g,prob.u0,(0.0,1.0))

reltols = 1.0./10.0.^(1:5)
abstols = reltols#[0.0 for i in eachindex(reltols)]

setups = [Dict(:alg=>SRIW1())
          Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 1))
          Dict(:alg=>SRIW1(),:dts=>1.0./5.0.^((1:length(reltols)) + 1),:adaptive=>false)
          ]
names = ["SRIW1","EM","RKMil","SRIW1 Fixed"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,names=names,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3.50 
 
 
 10 
 
 
 - 
 
 
 3.25 
 
 
 10 
 
 
 - 
 
 
 3.00 
 
 
 10 
 
 
 - 
 
 
 2.75 
 
 
 10 
 
 
 - 
 
 
 2.50 
 
 
 10 
 
 
 - 
 
 
 2.25 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 SRIW1 
 
 
 
 EM 
 
 
 
 RKMil 
 
 
 
 SRIW1 Fixed

Note that in this last problem, the adaptivity algorithm accurately detects that the error is already low enough, and does not increase the number of steps as the tolerance drops further.

In [11]:
setups = [Dict(:alg=>EM(),:dts=>1.0./5.0.^((1:length(reltols)) + 2))
          Dict(:alg=>RKMil(),:dts=>1.0./5.0.^((1:length(reltols)) + 2))
          Dict(:alg=>SRI())
          Dict(:alg=>SRIW1())
          Dict(:alg=>SRIW2())
          Dict(:alg=>SOSRI())
          Dict(:alg=>SOSRI2())
          ]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;numruns=1000,error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 8 
 
 
 10 
 
 
 - 
 
 
 7 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3.5 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 10 
 
 
 - 
 
 
 1.5 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 StochasticDiffEq.EM 
 
 
 
 StochasticDiffEq.RKMil 
 
 
 
 StochasticDiffEq.SRI 
 
 
 
 StochasticDiffEq.SRIW1 
 
 
 
 StochasticDiffEq.SRIW2 
 
 
 
 StochasticDiffEq.SOSRI 
 
 
 
 StochasticDiffEq.SOSRI2

### Conclusion

The RSwM3 adaptivity algorithm does not appear to have any significant overhead even on problems which do not necessitate adaptive timestepping. The tolerance clearly  In addition, the Rossler methods are shown to be orders of magnitude more efficient and should be used whenever applicable. The Oval2 tests show that these results are only magnified as the problem difficulty increases.